# Sequential removal of links and resiliency testing

### Overview
The notebook iteratively removes links (edges) in the FTOT road network in order of importance to create distinct disruption scenarios and re-runs FTOT to determine the new optimal solutions and costs. Importance is measured by:

- the sum of __betweenness centrality__ of a link's beginning and ending points __OR__   
- the __volume__ of background freight flows on a link.

The notebook outputs (i) CSV files with information on the links removed and corresponding scenario results and (ii) an interactive report generated with RMarkdown. Note that scenario costs in the outputs are the minimized FTOT objective value, which is based on impeded transport cost, facility build costs, and unmet demand penalties.

### Instructions
_Before running this notebook,_ follow instructions in the repository's README to (i) set up and activate the Python environment and (ii) run a baseline FTOT scenario.

__(1) Update parameters__ in the cell labeled Step 1:
- Baseline scenario name and path
- Measure of importance (volume or betweenness centrality)
- Number of disruption steps
- True/False toggle to export maps for each disruption scenario

__(2) Run all cells__ by going to the top menu bar > Cell > Run All.

__(3) Review outputs__ in the folder with disruption scenarios:

- Edges_to_Remove.csv - a list of edges to remove with their importance ranking
- Results.csv - resulting scenario costs and other optimal solution metrics after each disruption step
- Disruption_Results.html - interactive summary report

The disruption folder will be created in the same location as the baseline scenario folder. The new folder will have `V_disrupt` or `BC_disrupt` appended to the scenario name.

This notebook may take several hours to run depending on the scenario size and number of disruption steps.

### Troubleshooting HTML Report Creation
The final cells in the Conduct_Link_Removal.ipynb notebook generate the HTML report. If these cells error out and you do not see an HTML report pop up in your browser or appear in the disruption folder (located at the same place as your baseline scenario folder), please try the following steps:

1) Open a new Anaconda Prompt window and activate the environment by running `conda activate FTOTnetworkEnv`
2) Navigate to the link_removal subfolder: `cd C:\github\FTOT-Resilience-Link_Removal\link_removal`
3) Run the following: `Rscript compile_report.R <BASELINE SCENARIO FOLDER> <DISRUPT TYPE>`. For example, for Reference Scenario 7 and disruption type 'V' for volume, run `Rscript compile_report.R C:\FTOT\scenarios\reference_scenarios\rs7_capacity V`
4) If successful, the prompt window should print out "Output created: Disruption_Results.html".
5) Scroll back down to Step 5 and re-run each cell underneath "Step 5" one-by-one by clicking into each cell, then from the top menu bar running Cell > Run Cells. You can alternatively click into each cell and press Ctrl + Enter.
6) The report should appear in your browser window and should be saved in the disruption folder.

If needed, reach out to the FTOT team at FTOT-Team@dot.gov for assistance.

### Assumptions
- You are working in a Python 3.x environment for this notebook. Refer to the README in this repository for setup instructions.
- You have access to an ArcGIS license server.
- A baseline FTOT scenario was run with Network Density Reduction (NDR_On) set to False in the scenario XML file and with the modified `program` folder from this repository in place of the baseline FTOT `program` folder.

## Step 0: Load Dependencies

In [ ]:
import pandas as pd
import sqlalchemy
import networkx as nx
import os
import pickle
import subprocess
import shutil
import webbrowser
import resiliency_disruptions
from osgeo import ogr
import time

PYTHON = r"C:\FTOT\python3_env\python.exe"
FTOT = r"C:\FTOT\program\ftot.py"

## Step 1: Set User-Defined Parameters (USER INPUT REQUIRED)

In [ ]:
# Uses Reference Scenario 7 as an example.
# Modify `scen_name` and `scen_path` for your scenario.
scen_name = 'rs7_capacity'
scen_path = r'C:\FTOT\scenarios\reference_scenarios\rs7_capacity'

# Enter disrupt_type 'BC' for betweenness centrality or 'V' for volume.
# Note: If background flows were not enabled in the baseline scenario,
# the notebook will automatically switch to BC.
disrupt_type = 'V'

# Enter the number of disruption scenarios to generated.
# Recommend at least 25.
disrupt_steps = 25

# Set the variable `MAKE_MAPS` to `True` to you wish to output maps for each disruption scenario.
# Note this will increase runtime.
MAKE_MAPS = False

## Step 2: Calculate Importance Metrics

In [ ]:
if disrupt_type == 'BC':
    
    # Read in prepared betweenness centrality and road network graph data
    # If these don't exist, the following steps will create them
    picklename = os.path.join(scen_path, 'BetweenessG.pickle')
    if os.path.exists(picklename):
        file = open(picklename, 'rb')
        betweenness_dict_road = pickle.load(file)
        G_road = pickle.load(file)
    
    # Run betweenness centrality on the NetworkX graph
    # Note: This step might take several minutes to a few hours
    elif not os.path.exists(picklename):
        G_road = resiliency_disruptions.read_gdb(os.path.join(scen_path, 'main.gdb'), 'road')
        print('Running Betweenness Centrality calculations. This might take more than 20 minutes.')
        betweenness_dict_road = nx.betweenness_centrality(G_road, normalized=False, weight='Length')
        print('Completed Betweenness Centrality calculations.')
        
        # Save with pickle
        # Upon load, this pickle will contain the network G_road and the betweenness centrality dict
        with open(picklename, 'wb') as handle:
            pickle.dump(betweenness_dict_road, handle)
            pickle.dump(G_road, handle)

## Step 3: Associate Importance Metrics with Edges

In [ ]:
# Read in FTOT data
print('Reading in {}'.format(scen_path))

db_name = 'main.db'
db_path = 'sqlite:///' + os.path.join(scen_path, db_name)
engine = sqlalchemy.create_engine(db_path)

table_name = 'networkx_edges'
nx_edges = pd.read_sql_table(table_name, engine)

table_name = 'networkx_nodes'
nx_nodes = pd.read_sql_table(table_name, engine)

table_name = 'optimal_variables'
optimal_vars = pd.read_sql_table(table_name, engine)

In [ ]:
# Check whether scenario has background flow data
# Volume column in DB is filled with NULL if not
# Automatically revert to betweenness centrality if no background flow data
BACKGROUND_FLOWS = pd.isna(nx_edges['volume']).any()
if BACKGROUND_FLOWS:
    print('Background flows confirmed')
elif disrupt_type == 'V' and not BACKGROUND_FLOWS:
    print('WARNING: Network does not have background flows.')
    print('Switching importance measure to betweenness centrality.')
    disrupt_type = 'BC'
else:
    print('Scenario does not have background flows. Proceeding with disrupt type BC.')

In [ ]:
if disrupt_type == 'BC':
    
    # Get shape_x and shape_y
    road_orig_label_nodes = list(G_road.nodes)
    node_shape_df_road = pd.DataFrame(road_orig_label_nodes)
    
    # Make the betweenness_centrality values as the framework to join in shape_x, shape_y, and node_id
    bc_df_road = pd.DataFrame.from_dict(betweenness_dict_road, orient = 'index')
    bc_df_road = bc_df_road.rename(columns = {0: 'BC'}).reset_index()
    
    bc_shape_df_road = pd.concat([bc_df_road, node_shape_df_road], axis = 1)
    bc_shape_df_road = bc_shape_df_road.rename(columns = {0: 'shape_x', 1: 'shape_y'})
        
    # Now add node_id from networkx_nodes, using pandas merge with left join
    # Use both shape_x and shape_y to identify the nodes correctly
    bc_node_df = pd.merge(bc_shape_df_road, nx_nodes, on = ['shape_x', 'shape_y'], how = 'left')

    # Now use this dataframe to populate a dataframe of edges
    # We will want the following from networkx_edges:
    # edge_id, from_node_id, to_node_id, mode_source, miles, mode_source_oid
    # Then using the node_id column in the new bc_node_df, add these:
    # from_node_BC, to_node_BC
    # and sum those for sum_node_BC
    merge_from = pd.merge(nx_edges, bc_node_df[['BC', 'node_id']],
                          left_on = 'from_node_id',
                          right_on = 'node_id',
                          how = 'left')
    merge_from = merge_from.rename(columns = {'BC': 'from_node_BC'})

    merge_to = pd.merge(merge_from, bc_node_df[['BC', 'node_id']],
                        left_on = 'to_node_id',
                        right_on = 'node_id',
                        how = 'left')
    merge_to = merge_to.rename(columns = {'BC': 'to_node_BC'})

    # Sum the BC values
    merge_to['sum_BC'] = merge_to.filter(like = "node_BC").sum(axis = 1)

In [ ]:
if disrupt_type == 'V':
    merge_to = nx_edges.copy()

In [ ]:
# Select optimal_vars DB columns to keep
use_opt_vars = ['variable_type',
                'var_id',
                'variable_value',
                'variable_name',
                'nx_edge_id',
                'mode_oid',
                'converted_capacity',
                'converted_volume',
                'commodity_name'
               ]

merge_opt = pd.merge(merge_to, optimal_vars[use_opt_vars],
                     left_on = 'edge_id',
                     right_on = 'nx_edge_id',
                     how = 'left')

merge_opt.head()

In [ ]:
# Create ranked list of edges to remove
# (1) Keep only edges in the optimal solution
# (2) Sort by sum_BC or volume
# (3) Keep the columns we need
# (4) Reset the index to assign rank

# Note: in resiliency_disruptions.disrupt_network, the edges_remove DataFrame is sorted again by 'V' or 'BC'

use_cols = ['edge_id', 'from_node_id', 'to_node_id', 'length', 'capacity', 'volume', 'sum_BC',
            'variable_type', 'commodity_name', 'variable_value', 'variable_name', 'nx_edge_id', 'mode_oid', 'converted_capacity',
            'converted_volume']

if disrupt_type == 'V':
    edges_remove = merge_opt[merge_opt['variable_value'] > 0].sort_values(by = 'volume', ascending = False).filter(items = use_cols).reset_index()
elif disrupt_type == 'BC':
    edges_remove = merge_opt[merge_opt['variable_value'] > 0].sort_values(by = 'sum_BC', ascending = False).filter(items = use_cols).reset_index()

edges_remove.head()

In [ ]:
# Export list of edges to remove
disrupt_root = os.path.join(os.path.split(scen_path)[0],
                            '_'.join([os.path.split(scen_path)[1], disrupt_type, 'disrupt']))

if not os.path.exists(disrupt_root):
    os.mkdir(disrupt_root)

edges_remove.to_csv(os.path.join(disrupt_root, 'Edges_to_Remove.csv'), index = False)

## Step 4: Create Scenarios, Disrupt Edges, and Run FTOT

Create disrupted network by copying everything in `scen_path` to a new directory and overwriting the `networkx_edge_costs` table in the main.db with the disrupted versions of network edge cost.

##### Assumptions:

  1. ArcGIS Pro is installed and the license is accessible.
  2. The FTOT version being used has been modified according to the `README` in this directory.


In [ ]:
# Make new scenarios
resiliency_disruptions.make_disruption_scenarios(disrupt_type, disrupt_steps, scen_path)

# Apply disruptions
resiliency_disruptions.disrupt_network(disrupt_type, disrupt_steps, scen_path, edges_remove)

In [ ]:
# Run O through D (and optionally M) steps of FTOT on the disupted scenarios
# This may take several hours, depending on size of the network and number of disruption scenarios
results = resiliency_disruptions.run_o_steps(disrupt_type, disrupt_steps, scen_path, PYTHON, FTOT, MAKE_MAPS)

## Step 5: Generate Disruption Results Report

In [ ]:
here = os.getcwd()

# Delete existing HTML in repository if found
if os.path.exists(os.path.join(here, 'Disruption_Results.html')):
    os.remove(os.path.join(here, 'Disruption_Results.html'))

In [ ]:
# Render RMarkdown report
R_Process = subprocess.Popen(['Rscript.exe', 'compile_report.R', scen_path, disrupt_type],
                             stdout = subprocess.PIPE, stderr = subprocess.PIPE)

print ('Rendering report...')

# Allow report time to render
# Exit while loop if HTML report is found AND knit.md file is no longer found
# Or time out after 60 seconds
timer = 0
while timer <= 60 and (not os.path.exists(os.path.join(here, 'Disruption_Results.html')) or os.path.exists(os.path.join(here, 'Disruption_Results.knit.md'))):
    time.sleep(5) # Pause for 5 seconds
    timer += 5

# Move rendered HTML file when complete to the top-level disruption folder
# This will replace any existing file
if not os.path.exists(os.path.join(here, 'Disruption_Results.html')):
    print("OUTPUT FILE ERROR: Disruption_Results.html could not be found")
    raise Exception("OUTPUT FILE ERROR: Disruption_Results.html could not be found")

disrupt_root = scen_path + "_" + disrupt_type + "_disrupt"
new_html_location =  os.path.join(disrupt_root, 'Disruption_Results_' + disrupt_type + '_' + str(disrupt_steps) +'.html')
shutil.move(os.path.join(here, 'Disruption_Results.html'), new_html_location)

print('Finished: Report saved at: {}'.format(new_html_location))

In [ ]:
webbrowser.open('file://' + os.path.realpath(new_html_location))